#### Paradigm description:     
Experimental set-up:  

Gesture cue length : 2s  
f/b Black screen (resting state) : 2-3s  
  
rock-paper-scissor cue : 30 trials each

Sampling Frequency : 1200 Hz

60-channel high-density ECoG grids (Unique
Medical Co., Ltd.; diameter 1.5mm, spacing 5mm, geometry 6 ×
10)

In [1]:
import pandas as pd
import numpy as np
import scipy.io

import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt

import seaborn as sns

# !pip install mne
import mne

<b> 1. Importing Data

In [2]:
mat = scipy.io.loadmat('ECoG_Handpose.mat')

channels = [f'CH{i}' for i in range(1, 61)]
glove_data=['data_glove_thumb','data_glove_index','data_glove_middle','data_glove_ring','data_glove_little']
hand_gesture = ['paradigm_info']
cols = ['sample_time']+channels+hand_gesture+glove_data

ecog = pd.DataFrame.from_dict(mat['y'].T)
ecog.columns=cols
ecog.head()

,sample_time,CH1,CH2,CH3,CH4,CH5,CH6,CH7,CH8,CH9,...,CH57,CH58,CH59,CH60,paradigm_info,data_glove_thumb,data_glove_index,data_glove_middle,data_glove_ring,data_glove_little
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000833,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.001667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.002500,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.003333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<b> 2. Dropping initial non-experimental data

In [3]:
exp_start = ecog[ecog['paradigm_info'].ne(0)].index[0]
ecog.drop(index=list(range(0,exp_start-2400)),inplace=True)
ecog.reset_index(inplace=True, drop=True)

In [4]:
ecog['time'] = ecog.index / 1200
ecog.drop(columns='sample_time', inplace=True)
ecog.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 494929 entries, 0 to 494928
Data columns (total 67 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   CH1                494929 non-null  float64
 1   CH2                494929 non-null  float64
 2   CH3                494929 non-null  float64
 3   CH4                494929 non-null  float64
 4   CH5                494929 non-null  float64
 5   CH6                494929 non-null  float64
 6   CH7                494929 non-null  float64
 7   CH8                494929 non-null  float64
 8   CH9                494929 non-null  float64
 9   CH10               494929 non-null  float64
 10  CH11               494929 non-null  float64
 11  CH12               494929 non-null  float64
 12  CH13               494929 non-null  float64
 13  CH14               494929 non-null  float64
 14  CH15               494929 non-null  float64
 15  CH16               494929 non-null  float64
 16  CH

<b> 3. Creating mne info and raw data from ecog channels data

In [5]:
sampling_freq = 1200

info = mne.create_info(ch_names=channels+hand_gesture+glove_data,
                       ch_types=['ecog'] * 60+['stim']+['emg']*5,
                       sfreq=sampling_freq)

data=ecog[channels+hand_gesture+glove_data].T

raw = mne.io.RawArray(data, info)
raw

<ipython-input-5-346924feecd6>:5: RuntimeWarning: 4 channel names are too long, have been truncated to 15 characters:
['data_glove_thumb', 'data_glove_index', 'data_glove_middle', 'data_glove_little']
  sfreq=sampling_freq)


Creating RawArray with float64 data, n_channels=66, n_times=494929
    Range : 0 ... 494928 =      0.000 ...   412.440 secs
Ready.


<RawArray | 66 x 494929 (412.4 s), ~249.3 MB, data loaded>

---- Extract events from stimulus data (experimental cues) and plotting

In [6]:
# plt.close()
raw.copy().pick_types(emg=False, ecog=False, stim=True).plot(duration=200)
plt.show(block=False)

In [7]:
events = mne.find_events(raw)

90 events found
Event IDs: [1 2 3]


In [8]:
event_id = {'rock':1, 'paper': 2, 'scissor':3}

In [9]:
len(events[events[:,2]==1])

30

In [10]:
plt.close()
tags = ['relax', 'rock', 'paper', 'scissor']
paradigm_info = dict(zip(sorted(ecog['paradigm_info'].unique()), tags))
sns.countplot(x='paradigm_info', data=ecog, color='lightblue')
plt.show(block=False)

Channels marked as bad: none


In [11]:
plt.close()
fig = mne.viz.plot_events(events, event_id=event_id, sfreq=raw.info['sfreq'],
                          first_samp=raw.first_samp, show=False)
plt.show(block=False)

---- Plotting ECOG data

In [12]:
plt.close()
raw_ecog = raw.copy().pick_types(emg=False, ecog=True, stim=False)
raw_ecog.plot(duration=100, events=events, event_id=event_id, event_color = 'red', color='blue', scalings=dict(ecog=300))
# mne_raw.plot()
plt.show(block=False)

---- Plotting EMG glove data

In [13]:
plt.close()
raw_emg = raw.copy().pick_types(emg=True, ecog=False, stim=False)
raw_emg.plot(duration=100, events=events, event_id=event_id, event_color = 'red', color='blue',scalings=dict(emg=1))
# raw.plot()
plt.show(block=False)

Channels marked as bad: none


<b> 5. Re-referencing channels to common average

In [14]:
rereferenced_raw, ref_data = mne.set_eeg_reference(raw_ecog, ref_channels='average',ch_type='ecog',
                                                   copy=True)

Applying average reference.
Applying a custom ECoG reference.


In [15]:
plt.close('all')
rereferenced_raw.plot(duration=100, proj=False, scalings=dict(ecog=100),events=events, event_id=event_id,event_color = 'red', color='blue',
         n_channels=len(channels))

Channels marked as bad: none


<MNEBrowseFigure size 1520x1000 with 4 Axes>

<b> 6. Sinal pre-processing :</b>  
(recursive 6th-order Butterworth, bandwidth: 5 Hz)  up to the 6th harmonic was used to remove interference peaks from the spectrum at integer multiples of the power line frequency.

> <b>A.</b>  Applying notch-filter cascade to remove interference at power line frequency

In [16]:
plt.close()
rereferenced_raw.plot_psd(show=False) # Looking for power line frequency (interference peaks)
plt.title('original signal')
plt.show(block=False)

Channels marked as bad: none
Effective window size : 1.707 (s)


<ipython-input-16-103bef1bb718>:2: RuntimeWarning: Channel locations not available. Disabling spatial colors.
  rereferenced_raw.plot_psd(show=False) # Looking for power line frequency (interference peaks)


In [17]:
power_line_peaks = (50, 250, 350, 450, 550)

In [18]:
raw_notch = rereferenced_raw.copy().notch_filter(freqs=power_line_peaks)
raw_notch.plot_psd(show=False)
plt.title('After removing interference at power line frequency')
plt.show(block=False)

Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 7921 samples (6.601 sec)

Effective window size : 1.707 (s)


<ipython-input-18-53c5bb528653>:2: RuntimeWarning: Channel locations not available. Disabling spatial colors.
  raw_notch.plot_psd(show=False)


> <b>B.</b> Applying 6th order butterworth recursive filter bandpassed through high gamma frequency (50 Hz to 300 Hz)

In [19]:
from scipy import signal

iir_params = dict(order=6, ftype='butter',trans_bandwidth=5)
iir_params = mne.filter.construct_iir_filter(iir_params, f_pass=[50. , 300.],
                                             sfreq=sampling_freq, btype='bandpass', return_copy=True)

# filt = mne.filter.create_filter(raw_notch, sampling_freq, l_freq=50, h_freq=300,
#                                 method='iir', iir_params=iir_params)
# plot_filter(filt, sampling_freq, freq, gain, 'Butterworth order=8', flim=flim,
#             compensate=True)

filtered = raw_notch.copy().filter(l_freq=50, h_freq=300,
                                   picks='ecog', method='iir', 
                        iir_params=iir_params)

# filtered_ecog = signal.sosfiltfilt(filt['sos'], raw_notch)


IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 24 (effective, after forward-backward)
- Cutoffs at 50.00, 300.00 Hz: -6.02, -6.02 dB

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 50 - 3e+02 Hz



In [20]:
filtered.plot_psd(average=False,show=False)
plt.title('filtered signal')
plt.show(block=False)

Effective window size : 1.707 (s)


<ipython-input-20-f18634142e9f>:1: RuntimeWarning: Channel locations not available. Disabling spatial colors.
  filtered.plot_psd(average=False,show=False)


In [21]:
plt.close('all')
filtered.plot(duration=10, proj=False, scalings=dict(ecog=50),
              events=events, event_id=event_id,
              event_color = 'red', color='blue', n_channels=60)
plt.title('filtered  signal')
plt.show(block=False)

> <b>C.</b> Estimating the bandpower via a sliding variance window of 50 ms length, without overlap.

In [34]:
#help(mne.Epochs)

In [23]:
epochs = mne.Epochs(filtered, events, event_id=event_id, tmin=-0.2,tmax =2,preload=True)

Not setting metadata
Not setting metadata
90 matching events found
Setting baseline interval to [-0.2, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 90 events and 2641 original time points ...
0 bad epochs dropped


In [24]:
plt.close()
epochs.plot_image(show=False)
plt.show(block=False)

Channels marked as bad: none
Not setting metadata
Not setting metadata
90 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
combining channels using "gfp"


In [25]:
epochs.load_data()
alpha_data = epochs.get_data()

corr_matrix = mne.connectivity.envelope_correlation(alpha_data, combine=None)

corr_matrix.shape

first_event = corr_matrix[0]
last_event = corr_matrix[-1]
corr_matrices = [first_event, last_event]
color_lims = np.percentile(np.array(corr_matrices), [5, 95])
titles = ['First event', 'Last event']

fig, axes = plt.subplots(nrows=1, ncols=2)
fig.suptitle('Correlation Matrices from First and Last epoch')
for ci, corr_matrix in enumerate(corr_matrices):
    ax = axes[ci]
    mpbl = ax.imshow(corr_matrix, clim=color_lims)
    ax.set_xlabel(titles[ci])
fig.subplots_adjust(right=0.8)
# cax = fig.add_axes([0.85, 0.2, 0.025, 0.6])
# cbar = fig.colorbar(ax.images[0], cax=cax)
# cbar.set_label('Correlation Coefficient')
plt.show(block=False)

In [26]:
rock = epochs['rock']
paper = epochs['paper']
scissor = epochs['scissor']

In [27]:
plt.close('all')

rock.plot_image(show=False, title='Rock')
plt.show(block=False)

paper.plot_image(show=False,title='Paper')
plt.show(block=False)

scissor.plot_image(show=False, title='Scissor')
plt.show(block=False)

Not setting metadata
Not setting metadata
30 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
combining channels using "gfp"
Not setting metadata
Not setting metadata
30 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
combining channels using "gfp"
Not setting metadata
Not setting metadata
30 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
combining channels using "gfp"


In [28]:
rock_evoked = rock.average()
paper_evoked = paper.average()
scissor_evoked = scissor.average()

plt.close('all')
mne.viz.plot_compare_evokeds(dict(rock=rock_evoked, paper=paper_evoked, scissor = scissor_evoked),
                             legend='upper left', show_sensors='upper right', show=False)
plt.show(block=False)

combining channels using "gfp"
combining channels using "gfp"
combining channels using "gfp"


<ipython-input-28-119400c3abe5>:7: RuntimeWarning: Cannot find channel coordinates in the supplied Evokeds. Not showing channel locations.
  legend='upper left', show_sensors='upper right', show=False)


In [29]:
for event in paper:
    print(event[0].shape)
    break

(2641,)


In [30]:
for e in rock:
    print(e.shape)
    break

(60, 2641)


In [31]:
df = epochs.to_data_frame()

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 237690 entries, 0 to 237689
Data columns (total 63 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   time       237690 non-null  int64  
 1   condition  237690 non-null  object 
 2   epoch      237690 non-null  int64  
 3   CH1        237690 non-null  float64
 4   CH2        237690 non-null  float64
 5   CH3        237690 non-null  float64
 6   CH4        237690 non-null  float64
 7   CH5        237690 non-null  float64
 8   CH6        237690 non-null  float64
 9   CH7        237690 non-null  float64
 10  CH8        237690 non-null  float64
 11  CH9        237690 non-null  float64
 12  CH10       237690 non-null  float64
 13  CH11       237690 non-null  float64
 14  CH12       237690 non-null  float64
 15  CH13       237690 non-null  float64
 16  CH14       237690 non-null  float64
 17  CH15       237690 non-null  float64
 18  CH16       237690 non-null  float64
 19  CH17       237690 non-n

In [33]:
df.head()

,time,condition,epoch,CH1,CH2,CH3,CH4,CH5,CH6,CH7,...,CH51,CH52,CH53,CH54,CH55,CH56,CH57,CH58,CH59,CH60
0,-200,scissor,0,1.570628e+06,-769415.141292,-7.760798e+05,-3.786215e+06,-9.434679e+05,7.377976e+05,-9.163030e+06,...,-8.793391e+05,7.553653e+05,1.740024e+06,3.695847e+05,-6.738478e+06,4.544353e+06,1.483734e+06,2.357967e+06,1.955420e+06,1.155790e+06
1,-199,scissor,0,1.026301e+06,371.247055,1.944913e+05,-1.095369e+06,-2.137739e+05,-1.363444e+06,-6.495378e+06,...,-1.446691e+06,-9.835938e+05,-9.992447e+05,-4.194699e+06,-1.709372e+06,9.157686e+05,-1.552966e+05,2.522642e+06,2.763560e+06,1.342575e+06
2,-198,scissor,0,7.941045e+04,-13825.732957,1.414168e+06,2.144395e+06,1.447105e+06,-7.908181e+05,-3.368648e+06,...,-1.141727e+06,-2.994925e+06,-3.499307e+06,-6.941603e+06,4.459067e+06,-3.171257e+06,-2.690478e+06,1.123472e+06,2.126757e+06,1.124693e+06
3,-198,scissor,0,-5.013333e+05,-677346.077779,1.343599e+06,3.756443e+06,3.173911e+06,1.255666e+06,2.905267e+05,...,3.430999e+05,-4.161373e+06,-5.136841e+06,-7.661129e+06,8.174312e+06,-5.256110e+06,-4.649741e+06,7.563953e+04,1.545301e+06,1.349787e+06
4,-197,scissor,0,-2.130147e+06,-804821.948564,4.301506e+05,3.729284e+06,3.885446e+06,2.711634e+06,4.174270e+06,...,2.605582e+06,-4.063870e+06,-5.853666e+06,-7.104946e+06,8.485982e+06,-4.425439e+06,-4.604115e+06,6.290343e+05,1.833742e+06,2.086933e+06
